In [9]:
import pandas as pd
import datetime as dt

In [11]:
df = pd.read_json("data/Histórico de localização.json")

In [12]:
df.describe

<bound method NDFrame.describe of                                                  locations
0        {'timestampMs': '1378508750990', 'latitudeE7':...
1        {'timestampMs': '1378508766022', 'latitudeE7':...
2        {'timestampMs': '1378509026816', 'latitudeE7':...
3        {'timestampMs': '1378509072785', 'latitudeE7':...
4        {'timestampMs': '1378509118790', 'latitudeE7':...
...                                                    ...
1329815  {'timestampMs': '1635960303224', 'latitudeE7':...
1329816  {'timestampMs': '1635960483754', 'latitudeE7':...
1329817  {'timestampMs': '1635960664538', 'latitudeE7':...
1329818  {'timestampMs': '1635960844743', 'latitudeE7':...
1329819  {'timestampMs': '1635960914443', 'latitudeE7':...

[1329820 rows x 1 columns]>

In [13]:
locations = pd.json_normalize(df['locations'])

In [14]:
locations['timestampMs'] = pd.to_numeric(locations['timestampMs'])
locations['latitudeE7'] = pd.to_numeric(locations['latitudeE7'])
locations['longitudeE7'] = pd.to_numeric(locations['longitudeE7'])

In [15]:
locations = locations[locations['timestampMs'] > dt.datetime.timestamp(dt.datetime.now() - dt.timedelta(days=15))*1000]

In [16]:
import requests
import time

In [17]:
####
# URL de conexão com a API CEP Aberto
####
with open('token.txt') as file:
    token = file.readline()

url = "https://www.cepaberto.com/api/v3/nearest"
headers = {'Authorization': 'Token token=' + token}

In [18]:
entries = []
initTimestamp = 0
initLatitude = 0
initLongitude = 0
for index, location in locations.iterrows():
  if initTimestamp == 0:
    initTimestamp = location['timestampMs']
    initLatitude = location['latitudeE7']
    initLongitude = location['longitudeE7']
  else:
    ####
    # Verifica se houve alguma mudança significativa na localização baseada nos dados de lat e lng fornecidos pelo GPS
    ####
    if abs(location['latitudeE7'] - initLatitude) > 50000 or abs(location['longitudeE7'] - initLongitude) > 50000:
      duration = (dt.datetime.fromtimestamp(location['timestampMs']/1000) - dt.datetime.fromtimestamp(initTimestamp/1000)).total_seconds()
      ####
      # O usuário deve ter permanecido por pelo menos 30min no local para ser considerado
      ####
      if duration > 1800:
        ####
        # Formatando os dados de localização para se comunicar com a API do CEP Aberto
        ####
        latitude = location['latitudeE7']/10000000
        longitude = location['longitudeE7']/10000000
        response = requests.get(url, headers=headers, params={'lat': latitude, 'lng': longitude})
        cep = response.json()['cep']

        ####
        # Adicionando entrada à lista
        ####
        entries.append({'date' : dt.datetime.fromtimestamp(location['timestampMs']/1000).strftime("%Y/%m/%d, %H:%M:%S"),
                        'duration(s)' : duration,
                        'cep' : cep,
                        'latitude' : latitude,
                        'longitude' : longitude})
        
        ####
        # A API é free mas limitada à uma consulta por segundo
        ####
        time.sleep(1)
        
      initTimestamp = location['timestampMs']
      initLatitude = location['latitudeE7']
      initLongitude = location['longitudeE7']

In [19]:
len(entries)

18

In [20]:
entries

[{'date': '2021/10/22, 17:23:17',
  'duration(s)': 102338.422,
  'cep': '21815190',
  'latitude': -22.8821218,
  'longitude': -43.460417},
 {'date': '2021/10/24, 14:45:51',
  'duration(s)': 161701.578,
  'cep': '21810101',
  'latitude': -22.881368,
  'longitude': -43.4576364},
 {'date': '2021/10/24, 16:13:11',
  'duration(s)': 5118.068,
  'cep': '21715460',
  'latitude': -22.882494,
  'longitude': -43.449378},
 {'date': '2021/10/24, 17:40:18',
  'duration(s)': 4740.597,
  'cep': '21810101',
  'latitude': -22.8795846,
  'longitude': -43.4579725},
 {'date': '2021/10/25, 11:18:48',
  'duration(s)': 63449.34,
  'cep': '21810101',
  'latitude': -22.8812182,
  'longitude': -43.4581077},
 {'date': '2021/10/25, 19:23:56',
  'duration(s)': 15053.392,
  'cep': '21715340',
  'latitude': -22.8794555,
  'longitude': -43.4430027},
 {'date': '2021/10/25, 21:05:13',
  'duration(s)': 2139.678,
  'cep': '21820000',
  'latitude': -22.8792749,
  'longitude': -43.464273},
 {'date': '2021/10/25, 21:43:27',
